In [1]:
import numpy as np
import torch
import torch.nn as nn
from skimage.data import shepp_logan_phantom
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def cv2disp(name, image, xpos, ypos):
    cv2.imshow(name, image*1.0/(np.max(image)+1e-15))
    cv2.moveWindow(name, xpos, ypos)

In [3]:
def add_gaussian_noise(image, mean=0, std=25):
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image, noise

In [ ]:
#img = cv2.imread('lenna.png', cv2.IMREAD_GRAYSCALE)
#true_np = np.float64(img)/(1.0*np.max(img)+1e-15)
true_np = shepp_logan_phantom()
#noisy_np = np.random.normal(true_np)
noisy_np, noise_map = add_gaussian_noise(np.uint8(true_np*255), mean=0, std=1)
noisy_np = np.float64(noisy_np)/(1.0*np.max(noisy_np)+1e-15)
noise_map = np.float64(noise_map)/(1.0*np.max(noise_map)+1e-15)

nxd = true_np.shape[0]

_, axes = plt.subplots(1,2)
axes[0].imshow(true_np, cmap='gray')
axes[0].set_title('True')
axes[1].imshow(noisy_np, cmap='gray')
axes[1].set_title('Noisy')
plt.show()

In [ ]:
class CNN_NxN(nn.Module):
    def __init__(self, kernel_size):
        super(CNN_NxN, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, kernel_size, padding=(int(kernel_size/2), int(kernel_size/2)), bias=False)
        #self.conv1.weight.data.fill_(1.0/(kernel_size*kernel_size))
    def forward(self, x):
        x = self.conv1(x)
        return x

noisy_torch = torch.from_numpy(noisy_np).float().unsqueeze(0).unsqueeze(0)

cnn_fixed_kernel = CNN_NxN(31)

kernel = list(cnn_fixed_kernel.parameters())[0]
kernel_np = np.squeeze(kernel.detach().numpy())

cnn_out = cnn_fixed_kernel(noisy_torch)
cnn_out_np = np.squeeze(cnn_out.detach().numpy())

_, axes = plt.subplots(1,2)
axes[0].imshow(kernel_np, cmap='gray')
axes[0].set_title('Kernel')
axes[1].imshow(cnn_out_np, cmap='gray')
axes[1].set_title('CNN output')
plt.show()

In [6]:
# 1 layer CNN
cnn_trained_kernel = CNN_NxN(31)
loss_function = nn.MSELoss()
true_torch = torch.from_numpy(true_np).float().unsqueeze(0).unsqueeze(0)
optimizer = torch.optim.Adam(cnn_trained_kernel.parameters(), lr=0.003)

for epoch in range(500):
    output = cnn_trained_kernel(noisy_torch)
    loss = loss_function(output, true_torch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    kernel = list(cnn_trained_kernel.parameters())[0]
    cv2disp('Kernel', cv2.resize(np.squeeze(kernel.detach().numpy()), (nxd,nxd), interpolation=0), 0, 0)

    cnn_output = np.squeeze(output.detach().numpy())
    cv2.putText(cnn_output, 'Epoch %d' % epoch, (2,30), 0, 1, int(np.max(cnn_output)+1), 1, cv2.LINE_AA)
    cv2disp('Output after conv', cnn_output, nxd, 0)
    cv2.waitKey(1)

In [ ]:
# learn the clean image
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.CNN = nn.Sequential(
            nn.Conv2d(1, 8, 7, padding=(3,3)), nn.PReLU(),
            nn.Conv2d(8, 8, 7, padding=(3,3)), nn.PReLU(),
            nn.Conv2d(8, 8, 7, padding=(3,3)), nn.PReLU(),
            nn.Conv2d(8, 8, 7, padding=(3,3)), nn.PReLU(),
            nn.Conv2d(8, 1, 7, padding=(3,3)), nn.PReLU(),
        )
    def forward(self, x):
        x = self.CNN(x)
        return x

cnn_to_train = CNN()
loss_function = nn.MSELoss()
true_torch = torch.from_numpy(true_np).float().unsqueeze(0).unsqueeze(0)
optimizer = torch.optim.Adam(cnn_to_train.parameters(), lr=0.003)

for epoch in range(500):
    output = cnn_to_train(noisy_torch)
    loss = loss_function(output, true_torch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    cnn_output = np.squeeze(output.detach().numpy())
    cv2.putText(cnn_output, 'Epoch %d' % epoch, (2,30), 0, 1, int(np.max(cnn_output)+1), 1, cv2.LINE_AA)
    cv2disp('Output after conv', cnn_output, 0, 0)
    cv2.waitKey(1)